In [6]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://pp

In [7]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("vineReview").getOrCreate()

In [12]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Furniture_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Furniture_v1_00.tsv.gz"), sep="\t", header=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24509695|R3VR960AHLFKDV|B004HB5E0E|     488241329|Shoal Creek Compu...|       Furniture|          4|            0|          0|   N|                Y|... desk is very ...|This desk is very...| 2015-08-31|
|         US|   34731776|R16LGVMFKIUT0G|B0042TNMMS|     205864445|Dorel Home Produc...|       Furniture|          5|    

In [18]:
# create a vine_table with select columns
vine_table = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3VR960AHLFKDV|          4|            0|          0|   N|                Y|
|R16LGVMFKIUT0G|          5|            0|          0|   N|                Y|
|R1AIMEEPYHMOE4|          5|            1|          1|   N|                Y|
|R1892CCSZWZ9SR|          3|            0|          0|   N|                Y|
|R285P679YWVKD1|          3|            0|          0|   N|                N|
| RLB33HJBXHZHU|          5|            0|          0|   N|                Y|
|R1VGTZ94DBAD6A|          5|            2|          2|   N|                Y|
|R168KF82ICSOHD|          5|            0|          0|   N|                Y|
|R20DIYIJ0OCMOG|          5|            0|          0|   N|                Y|
| RD46RNVOHNZSC|          5|            0|          0|   N|     

In [19]:
# filter for total_votes greater than or equal to 20
vine_table_new = vine_table.filter(vine_table["total_votes"]>=20)
vine_table_new.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
|R3GNSIFV1J2Y2B|          1|           15|         60|   N|                N|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|     

In [20]:
# create a new DataFrame to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
filtered_vine_df = vine_table_new.filter(vine_table_new["helpful_votes"]/vine_table_new["total_votes"]>=0.5)
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|                Y|
| RJNDSWES5ISZ7|          5|           78|         79|   N|     

In [21]:
# retrieve all the rows where a review was written as part of the Vine program (paid)
paid_vine_df = filtered_vine_df.filter(filtered_vine_df.vine == "Y")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2BQOD1R0228FN|          3|           17|         26|   Y|                N|
| RC31RUPFOHBHQ|          4|          102|        117|   Y|                N|
| REN3N1WITLF1Y|          5|           33|         37|   Y|                N|
| R71RZQ9UZVG47|          4|           38|         47|   Y|                N|
|R38NMQBH88HLM6|          4|           18|         24|   Y|                N|
|R33FGX9EE3QVR6|          4|           26|         26|   Y|                N|
|R1KIOIK6WEYE59|          3|           19|         20|   Y|                N|
|R25X9BMOB3FD0E|          4|           32|         37|   Y|                N|
|R3VCKFCX2377Q2|          4|           95|        101|   Y|                N|
|R1E0OUG63HMSM4|          3|           58|         61|   Y|     

In [22]:
# retrieve all the rows where the review was not part of the Vine program (unpaid)
unpaid_vine_df = filtered_vine_df.filter(filtered_vine_df.vine == "N")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RL8D0KJ0J9L0O|          5|          152|        165|   N|                Y|
|R1BEINAIQFBRJC|          5|           21|         23|   N|                Y|
|R2L59KIJH302P9|          4|           26|         26|   N|                Y|
| RR99CPG695T0I|          5|          215|        248|   N|                N|
|R1XQNKKUPCMWVO|          5|           43|         44|   N|                Y|
|R3JUXVCT1NSK2A|          3|           25|         26|   N|                Y|
| RTCRZARYY4LXX|          5|           52|         54|   N|                Y|
|R3OFB4P7Y8WR27|          1|           15|         26|   N|                Y|
|R3MTAYGQM25N63|          4|           58|         59|   N|                Y|
| RJNDSWES5ISZ7|          5|           78|         79|   N|     

In [24]:
# total number of paid reviews
paid_total = paid_vine_df.count()
paid_total

136

In [25]:
# number of 5-star paid reviews
paid_5star = paid_vine_df.filter(paid_vine_df["star_rating"]==5).count()
paid_5star

74

In [27]:
# pecentage of paid 5-star reviews
paid_percentage = paid_5star/paid_total*100
paid_percentage

54.41176470588235

In [28]:
# total number of unpaid reviews
unpaid_total = unpaid_vine_df.count()
unpaid_total

18019

In [29]:
# number of 5-star unpaid reviews
unpaid_5star = unpaid_vine_df.filter(unpaid_vine_df["star_rating"]==5).count()
unpaid_5star

8482

In [30]:
# pecentage of unpaid 5-star reviews
unpaid_percentage = unpaid_5star/unpaid_total*100
unpaid_percentage

47.072534546867196